# GAN


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 
import numpy as np
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
from torchvision.utils import save_image

/DYZ/dyz1/anaconda3/envs/cv_project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = datasets.MNIST("./datasets/", 
                            download=True, 
                            train=True, 
                            transform=transforms.Compose([
                               transforms.ToTensor(), transforms.Normalize([0.5], [0.5])
                            ]))

test_data = datasets.MNIST('./datasets/', 
                           train=False, 
                           download=True, 
                           transform=transforms.Compose([
                              transforms.ToTensor(), transforms.Normalize([0.5], [0.5])
                            ]))

In [3]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [4]:
x,y = iter(train_loader).next()
print(x.shape)

torch.Size([64, 1, 28, 28])


In [5]:
## ##### 定义判别器 Discriminator ######
## 将图片28x28展开成784，然后通过多层感知器，中间经过斜率设置为0.2的LeakyReLU激活函数，
## 最后接sigmoid激活函数得到一个0到1之间的概率进行二分类    
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.discriminator = nn.Sequential(
            nn.Linear(784, 512),  ## 输入特征数为784，输出为512
            nn.LeakyReLU(0.2, inplace=True), ## 进行非线性映射
            nn.Linear(512, 256), ## 输入特征数为512，输出为256
            nn.LeakyReLU(0.2, inplace=True), ## 进行非线性映射
            nn.Linear(256, 1),  ## 输入特征数为256，输出为1
            nn.Sigmoid(), ## sigmoid是一个激活函数，二分类问题中可将实数映射到[0, 1],作为概率值, 多分类用softmax函数
        )
        
    def forward(self, 
                x:torch.Tensor):
        """_summary_
        Args:
            x (torch.Tensor): 
        Returns:
            _type_: 
        """
        x = x.flatten(1)
        return self.discriminator(x)

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        def block(in_feat, out_feat, normalize=True):
            layer = [nn.Linear(in_feat, out_feat)]          ## 线性变换将输入映射到out维
            if normalize:
                layer.append(nn.BatchNorm1d(out_feat, 0.8)) ## 正则化
            layer.append(nn.LeakyReLU(0.2, inplace=True)) ## 非线性激活函数
            return layer

        self.generator = nn.Sequential(
            *block(100, 128,normalize=False), ## 线性变化将输入映射 100 to 128, 正则化, LeakyReLU
            *block(128, 256),   ## 线性变化将输入映射 128 to 256, 正则化, LeakyReLU
            *block(256, 512),   ## 线性变化将输入映射 256 to 512, 正则化, LeakyReLU
            *block(512, 1024),  ## 线性变化将输入映射 512 to 1024, 正则化, LeakyReLU
            nn.Linear(1024, 784), ## 线性变化将输入映射 1024 to 784
            nn.Tanh() # ## 将(784)的数据每一个都映射到[-1, 1]之间
        )

    def forward(self, z):
        imgs = self.generator(z)                                     ## 噪声数据通过生成器模型
        imgs = imgs.view(imgs.size(0), 1, 28, 28)                ## reshape成(64, 1, 28, 28)
        return imgs                                              ## 输出为64张大小为(1, 28, 28)的图像  


In [6]:
## 参数
epcohs = 10
device = torch.device("cuda")

## 首先需要定义loss的度量方式  （二分类的交叉熵）
criterion = torch.nn.BCELoss()

## 创建生成器，判别器对象
generator = Generator()
discriminator = Discriminator()


## 其次定义 优化函数,优化函数的学习率为0.0003
## betas:用于计算梯度以及梯度平方的运行平均值的系数
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.003, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.003, betas=(0.5, 0.999))

## 都在cuda模式中运行
generator = generator.to(device)
discriminator = discriminator.to(device)
criterion = criterion.to(device)


In [11]:
## 训练
for epoch in range(epcohs):
    
    d_epoch_loss = 0
    g_epoch_loss = 0
    count = len(train_loader)  #返回批次数

    for i, (x, y) in enumerate(train_loader):
        bz = x.shape[0]
        x = x.to(device)
        ## ---------------------
        ##  Train Discriminator
        ## 分为两部分：1、真的图像判别为真；2、假的图像判别为假
        ## ---------------------
        ## 计算真实图片的损失
        real_out = discriminator(x)
        d_real_loss = criterion(real_out, torch.ones_like(real_out))
        
        ## 计算假的图片的损失
        ## detach(): 从当前计算图中分离下来避免梯度传到G，因为G不用更新
        random_noise = torch.randn(bz,100,device=device)
        gen_img = generator(random_noise)
        fake_output = discriminator(gen_img.detach()) 
        d_fake_loss = criterion(fake_output,torch.zeros_like(fake_output))
        
        d_loss = d_real_loss + d_fake_loss ## 损失包括判真损失和判假损失
        optimizer_D.zero_grad()  ## 在反向传播之前，先将梯度归0
        d_loss.backward()  ## 将误差反向传播
        optimizer_D.step() ## 更新参数
        
        d_epoch_loss += d_loss.item()
        
        ## -----------------
        ##  Train Generator
        ## 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        ## 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        ## 反向传播更新的参数是生成网络里面的参数，
        ## 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的, 这样就达到了对抗的目的
        ## -----------------

        random_noise = torch.randn(bz,100,device=device)
        fake_img = generator(random_noise)                                             ## 随机噪声输入到生成器中，得到一副假的图片
        output = discriminator(fake_img)                                    ## 经过判别器得到的结果
        ## 损失函数和优化
        g_loss = criterion(output, torch.ones_like(output))                             ## 得到的假的图片与真实的图片的label的loss
        optimizer_G.zero_grad()                                             ## 梯度归0
        g_loss.backward()                                                   ## 进行反向传播
        optimizer_G.step()                                                  ## step()一般用在反向传播后面,用于更新生成网络的参数

        g_epoch_loss += g_loss.item()
        
        ## 打印训练过程中的日志
        ## item():取出单元素张量的元素值并返回该值，保持原元素类型不变
        if (i + 1) % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [D real: %f] [D fake: %f]"
                % (epoch, 10, i, len(train_data), d_loss.item(), g_loss.item(), real_out.data.mean(), fake_output.data.mean())
            )
        # ## 保存训练过程中的图像

        if i % 500 == 0:
            batches_done = "_"+str(epoch)+"_"+ str(i)+"_"
            save_image(fake_img.data[:25], "./results/gan/images/%s.png" % batches_done, nrow=5, normalize=True)

    print("d_epoch_loss:", d_epoch_loss/count)
    print("g_epoch_loss:", g_epoch_loss/count)
        
## 保存模型
torch.save(generator.state_dict(), './results/gan/generator.pth')
torch.save(discriminator.state_dict(), './results/gan/discriminator.pth')       
        
        

[Epoch 0/10] [Batch 99/60000] [D loss: 1.057642] [G loss: 2.896710] [D real: 0.634835] [D fake: 0.448564]
[Epoch 0/10] [Batch 199/60000] [D loss: 0.699268] [G loss: 2.513327] [D real: 0.727830] [D fake: 0.265682]
[Epoch 0/10] [Batch 299/60000] [D loss: 1.090155] [G loss: 1.115966] [D real: 0.498161] [D fake: 0.298167]
[Epoch 0/10] [Batch 399/60000] [D loss: 1.131881] [G loss: 0.999511] [D real: 0.695840] [D fake: 0.525407]
[Epoch 0/10] [Batch 499/60000] [D loss: 2.041314] [G loss: 0.269469] [D real: 0.156802] [D fake: 0.075694]
[Epoch 0/10] [Batch 599/60000] [D loss: 0.927922] [G loss: 1.115815] [D real: 0.575406] [D fake: 0.293345]
[Epoch 0/10] [Batch 699/60000] [D loss: 0.957753] [G loss: 2.605673] [D real: 0.670651] [D fake: 0.404052]
[Epoch 0/10] [Batch 799/60000] [D loss: 0.925860] [G loss: 1.181737] [D real: 0.561867] [D fake: 0.270989]
[Epoch 0/10] [Batch 899/60000] [D loss: 1.073038] [G loss: 1.098074] [D real: 0.547136] [D fake: 0.365166]
d_epoch_loss: 1.301612191680652
g_epoc

In [ ]:
## ----------
##  Training
## ----------
## 进行多个epoch的训练
for epoch in range(10):                               ## epoch:50
    for i, (imgs, _) in enumerate(train_loader):                  ## imgs:(64, 1, 28, 28)     _:label(64)
        
        ## =============================训练判别器==================
        ## view(): 相当于numpy中的reshape，重新定义矩阵的形状, 相当于reshape(128，784)  原来是(128, 1, 28, 28)
        # imgs = imgs.view(imgs.size(0), -1)                          ## 将图片展开为28*28=784  imgs:(64, 784)
        real_img = Variable(imgs).cuda()                            ## 将tensor变成Variable放入计算图中，tensor变成variable之后才能进行反向传播求梯度
        real_label = Variable(torch.ones(imgs.size(0), 1)).cuda()      ## 定义真实的图片label为1
        fake_label = Variable(torch.zeros(imgs.size(0), 1)).cuda()     ## 定义假的图片的label为0


        ## ---------------------
        ##  Train Discriminator
        ## 分为两部分：1、真的图像判别为真；2、假的图像判别为假
        ## ---------------------
        ## 计算真实图片的损失
        real_out = discriminator(real_img)                          ## 将真实图片放入判别器中
        loss_real_D = criterion(real_out, real_label)               ## 得到真实图片的loss
        real_scores = real_out                                      ## 得到真实图片的判别值，输出的值越接近1越好
        ## 计算假的图片的损失
        ## detach(): 从当前计算图中分离下来避免梯度传到G，因为G不用更新
        z = Variable(torch.randn(imgs.size(0), 100)).cuda()      ## 随机生成一些噪声, 大小为(128, 100)
       
        fake_img = generator(z).detach()                                    ## 随机噪声放入生成网络中，生成一张假的图片。 
        fake_out = discriminator(fake_img)                                  ## 判别器判断假的图片
        loss_fake_D = criterion(fake_out, fake_label)                       ## 得到假的图片的loss
        fake_scores = fake_out                                              ## 得到假图片的判别值，对于判别器来说，假图片的损失越接近0越好
        ## 损失函数和优化
        loss_D = loss_real_D + loss_fake_D                  ## 损失包括判真损失和判假损失
        optimizer_D.zero_grad()                             ## 在反向传播之前，先将梯度归0
        loss_D.backward()                                   ## 将误差反向传播
        optimizer_D.step()                                  ## 更新参数


        ## -----------------
        ##  Train Generator
        ## 原理：目的是希望生成的假的图片被判别器判断为真的图片，
        ## 在此过程中，将判别器固定，将假的图片传入判别器的结果与真实的label对应，
        ## 反向传播更新的参数是生成网络里面的参数，
        ## 这样可以通过更新生成网络里面的参数，来训练网络，使得生成的图片让判别器以为是真的, 这样就达到了对抗的目的
        ## -----------------
        z = Variable(torch.randn(imgs.size(0), 100)).cuda()      ## 得到随机噪声
        fake_img = generator(z)                                             ## 随机噪声输入到生成器中，得到一副假的图片
        output = discriminator(fake_img)                                    ## 经过判别器得到的结果
        ## 损失函数和优化
        loss_G = criterion(output, real_label)                              ## 得到的假的图片与真实的图片的label的loss
        optimizer_G.zero_grad()                                             ## 梯度归0
        loss_G.backward()                                                   ## 进行反向传播
        optimizer_G.step()                                                  ## step()一般用在反向传播后面,用于更新生成网络的参数




        ## 打印训练过程中的日志
        ## item():取出单元素张量的元素值并返回该值，保持原元素类型不变
        if (i + 1) % 100 == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [D real: %f] [D fake: %f]"
                % (epoch, 10, i, len(train_loader), loss_D.item(), loss_G.item(), real_scores.data.mean(), fake_scores.data.mean())
            )